<a href="https://colab.research.google.com/github/jsaulme/Aprendizaje_Automatico_MCD/blob/main/fifa_worldcup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from string import ascii_uppercase as alfabeto
import pickle
from bs4 import BeautifulSoup
import requests

In [2]:
# URL con la infomacion previa a que se juegue el mundial de 2022
url = "https://web.archive.org/web/20221114220634/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup"

# Lee todas las tablas de la página web de Wikipedia y las almacena en una lista
todas_tablas = pd.read_html(url)
todas_tablas

[      0     1     2
 0   Oct   NOV   Dec
 1   NaN    14   NaN
 2  2021  2022  2023,
   2022 كأس العالم لكرة القدم Kaʾs al-ʿālam li-kurat al-qadam 2022 Qatar 2022 2022 قطر  \
 0                                         Now is All                                    
 1                                 Tournament details                                    
 2                                       Host country                                    
 3                                              Dates                                    
 4                                              Teams                                    
 5                                           Venue(s)                                    
 6                                      ← 2018 2026 →                                    
 
   2022 كأس العالم لكرة القدم Kaʾs al-ʿālam li-kurat al-qadam 2022 Qatar 2022 2022 قطر.1  
 0                                         Now is All                                     
 1         

In [3]:
# URL co la informacion posterior a que se jugo el mundial
#url_post = "https://en.wikipedia.org/wiki/2022_FIFA_World_Cup"
#data2026 = "https://web.archive.org/web/20221114220634/https://en.wikipedia.org/wiki/2026_FIFA_World_Cup"

In [4]:
alfabeto

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [5]:
# Itera sobre la lista de letras del alfabeto y un rango de números específicos
# 'zip' empareja cada letra con un número del rango (11, 68, 7) que va de 11 a 67 con incrementos de 7
for letra, i in zip(alfabeto, range(12, 65, 7)):
    print(letra, i)

A 12
B 19
C 26
D 33
E 40
F 47
G 54
H 61


In [6]:
# Grupo A -> H
todas_tablas[12] #GrupoA
#todas_tablas[19] #GrupoB
#todas_tablas[26] #GrupoC
#todas_tablas[33] #GrupoD
#todas_tablas[40] #GrupoE
#todas_tablas[47] #GrupoF
#todas_tablas[54] #GrupoG
#todas_tablas[61] #GrupoH

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Qatar (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ecuador,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Senegal,0,0,0,0,0,0,0,0,NaN
3,4,Netherlands,0,0,0,0,0,0,0,0,NaN


In [7]:
for i in range(12, 65, 7):
    print(todas_tablas[i].columns[1])

Teamvte
Teamvte
Teamvte
Teamvte
Teamvte
Teamvte
Teamvte
Teamvte


In [8]:
# Renombrar todas las columnas 1 por el nombre "Team" y elimina la columna "Qualification"
# Inicializa un diccionario vacío para almacenar las tablas procesadas.

dict_tablas = {}
for letra, i in zip(alfabeto, range(12, 68, 7)):  # Adjusted range to start from 12
    df = todas_tablas[i]

    # Check if the DataFrame has more than one column before renaming
    if len(df.columns) > 1:
        df.rename(columns={df.columns[1]: "Team"}, inplace=True)

    if "Qualification" in df.columns:  # Check if "Qualification" column exists before popping
        df.pop("Qualification")

    dict_tablas[f'Grupo {letra}'] = df

In [9]:
dict_tablas.keys()

dict_keys(['Grupo A', 'Grupo B', 'Grupo C', 'Grupo D', 'Grupo E', 'Grupo F', 'Grupo G', 'Grupo H'])

In [10]:
# dict_tablas['Grupo A']
# dict_tablas['Grupo B']
# dict_tablas['Grupo C']
# dict_tablas['Grupo D']
# dict_tablas['Grupo E']
# dict_tablas['Grupo F']
# dict_tablas['Grupo G']
dict_tablas['Grupo H']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Portugal,0,0,0,0,0,0,0,0
1,2,Ghana,0,0,0,0,0,0,0,0
2,3,Uruguay,0,0,0,0,0,0,0,0
3,4,South Korea,0,0,0,0,0,0,0,0


In [11]:
with open('dict_tablas', 'wb') as output:
    pickle.dump(dict_tablas, output)

In [12]:
# Ver archivos del entorno
!ls

dict_tablas  sample_data


In [13]:
# Set "years" que vaya desde 1930 a 2022 (incluido), de 4 en 4
years = set(range(1930, 2023, 4))
print(years)

{1930, 1934, 1938, 1942, 1946, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018, 2022}


In [14]:
# Funcion para obtener los partidos por año
def get_partidos(year):

  web = f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"    # URL con resultados de los partidos
  response = requests.get(web)
  # url = f"https://web.archive.org/web/20221114220634/https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"    #URL sin resultados de los partidos
  # response = requests.get(url)
  content = response.text
  soup = BeautifulSoup(content, 'lxml')
  partidos = soup.find_all("div", class_="footballbox")

  home = []
  score = []
  away = []

  for partido in partidos:
    home.append(partido.find("th", class_="fhome").get_text())
    score.append(partido.find("th", class_="fscore").get_text())
    away.append(partido.find("th", class_="faway").get_text())

  dict_partidos = {'Home': home, 'Score': score, 'Away': away}
  df_partidos = pd.DataFrame(dict_partidos)
  df_partidos["Year"] = year
  return df_partidos

In [15]:
print(get_partidos(1982))

               Home         Score               Away  Year
0            Italy            0–0             Poland  1982
1             Peru            0–0           Cameroon  1982
2            Italy            1–1               Peru  1982
3           Poland            0–0           Cameroon  1982
4           Poland            5–1               Peru  1982
5            Italy            1–1           Cameroon  1982
6     West Germany            1–2            Algeria  1982
7            Chile            0–1            Austria  1982
8     West Germany            4–1              Chile  1982
9          Algeria            0–2            Austria  1982
10         Algeria            3–2              Chile  1982
11    West Germany            1–0            Austria  1982
12       Argentina            0–1            Belgium  1982
13         Hungary           10–1        El Salvador  1982
14       Argentina            4–1            Hungary  1982
15         Belgium            1–0        El Salvador  19

In [16]:
# Obtiene los DataFrames de partidos para cada año en 'years'.
partidos_mundiales = [get_partidos(year) for year in years]

# Combina todos los DataFrames en uno solo, reiniciando los índices.
df_partidos_mundiales = pd.concat(partidos_mundiales, ignore_index=True)

# Guarda el DataFrame combinado en un archivo CSV.
df_partidos_mundiales.to_csv("fifa_mundiales_data.csv", index=False)


In [17]:
# Funcion para obtener el rol del mundial
def get_fixture(year):

  # web = f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"    # URL con resultados de los fixture
  # response = requests.get(web)
  url = f"https://web.archive.org/web/20221114220634/https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"    #URL sin resultados de los fixture
  response = requests.get(url)
  content = response.text
  soup = BeautifulSoup(content, 'lxml')
  fixture = soup.find_all("div", class_="footballbox")

  home = []
  score = []
  away = []

  for partido in fixture:
    home.append(partido.find("th", class_="fhome").get_text())
    score.append(partido.find("th", class_="fscore").get_text())
    away.append(partido.find("th", class_="faway").get_text())

  dict_fixture = {'Home': home, 'Score': score, 'Away': away}
  df_fixture = pd.DataFrame(dict_fixture)
  df_fixture["Year"] = year
  return df_fixture

In [18]:
# Extraer el rol de juegos (fixture) sin resultados del Mundial de 2022
df_fixture2022 = get_fixture(2022)

df_fixture2022

#df_fixture2022.to_csv("fifa_fixture_2022.csv", index=False)

,Home,Score,Away,Year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


In [151]:
# Renombrar 'Qatar' por 'Qatar (H)' en las columnas relevantes de df_fixture2022
df_fixture2022['Home'] = df_fixture2022['Home'].replace('Qatar', 'Qatar (H)')
df_fixture2022['Away'] = df_fixture2022['Away'].replace('Qatar', 'Qatar (H)')

df_fixture2022

,Home,Score,Away,Year
0,Qatar (H),Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar (H),Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


In [19]:
# lista todos los archivos y carpetas en el directorio actual
!ls

dict_tablas  fifa_mundiales_data.csv  sample_data


In [20]:
!google-chrome --version

/bin/bash: line 1: google-chrome: command not found


In [21]:
# @title
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb

--2024-08-11 19:39:19--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.16.91, 142.251.16.93, 142.251.16.136, ...
Connecting to dl.google.com (dl.google.com)|142.251.16.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109464716 (104M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 104.39M   178MB/s    in 0.6s    

2024-08-11 19:39:19 (178 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [109464716/109464716]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libu2f-udev libudev1 libvulkan1 mesa-vulkan-drivers systemd-hwe-hwdb udev
The following NEW packages will be installed:
  google-chrome-stable lib

In [22]:
!pip install selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)
driver.get('https://www.google.com')
print(driver.title)
#driver.quit()

Google


In [24]:
# Llama a la función get_partidos con el año 1982 y muestra el DataFrame resultante
print(get_partidos(1982))

               Home         Score               Away  Year
0            Italy            0–0             Poland  1982
1             Peru            0–0           Cameroon  1982
2            Italy            1–1               Peru  1982
3           Poland            0–0           Cameroon  1982
4           Poland            5–1               Peru  1982
5            Italy            1–1           Cameroon  1982
6     West Germany            1–2            Algeria  1982
7            Chile            0–1            Austria  1982
8     West Germany            4–1              Chile  1982
9          Algeria            0–2            Austria  1982
10         Algeria            3–2              Chile  1982
11    West Germany            1–0            Austria  1982
12       Argentina            0–1            Belgium  1982
13         Hungary           10–1        El Salvador  1982
14       Argentina            4–1            Hungary  1982
15         Belgium            1–0        El Salvador  19

In [25]:
# URL de la página web del Mundial de 1982.
web1982 = ("https://en.wikipedia.org/wiki/1982_FIFA_World_Cup")

# Navega a la página web especificada usando el controlador web (driver).
driver.get(web1982)

# Obtiene el título de la página web y lo imprime.
print(driver.title)

1982 FIFA World Cup - Wikipedia


In [26]:
element_italy = driver.find_element(by="xpath", value='//th[@class="fhome"]/span[@itemprop="name"]/a')
home_team = element_italy.text
print(home_team)  # Debería imprimir "Italy"

Italy


In [27]:
element_score = driver.find_element(by="xpath", value='//th[@class="fscore"]')
score = element_score.text
print(score)  # Debería imprimir "0-0"

0–0


In [28]:
# Extraer "Poland"
element_poland = driver.find_element(by="xpath", value='//th[@class="faway"]//span[@itemprop="name"]//a')
away_team = element_poland.text
print(away_team)  # Debería imprimir "Poland"


Poland


In [29]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# # Función para obtener los partidos por año
# def get_partidos_faltantes(year):
#     web = f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"
#     response = requests.get(web)
#     content = response.text
#     soup = BeautifulSoup(content, 'lxml')

#     # Encontrar todos los partidos
#     partidos = soup.find_all("div", class_="footballbox")

#     home = []
#     score = []
#     away = []

#     for partido in partidos:
#         # Extraer equipo local
#         home_team = partido.find("th", class_="fhome").get_text().strip()
#         home.append(home_team)

#         # Extraer resultado
#         match_score = partido.find("th", class_="fscore").get_text().strip()
#         score.append(match_score)

#         # Extraer equipo visitante
#         away_team = partido.find("th", class_="faway").get_text().strip()
#         away.append(away_team)

#     # Crear un diccionario con los resultados
#     dict_partidos = {'Home': home, 'Score': score, 'Away': away}
#     df_partidos = pd.DataFrame(dict_partidos)
#     df_partidos["Year"] = year
#     return df_partidos

# # Usar la función para obtener los partidos de 1982
# df_partidos_1982 = get_partidos(1982)
# print(df_partidos_1982)

In [30]:
# Función para obtener los partidos por año
def get_partidos_by_year(year):
    web = f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')

    # Encontrar todos los partidos
    partidos = soup.find_all("div", class_="footballbox")

    home = []
    score = []
    away = []

    for partido in partidos:
        # Extraer equipo local
        home_team = partido.find("th", class_="fhome").get_text().strip()
        home.append(home_team)

        # Extraer resultado
        match_score = partido.find("th", class_="fscore").get_text().strip()
        score.append(match_score)

        # Extraer equipo visitante
        away_team = partido.find("th", class_="faway").get_text().strip()
        away.append(away_team)

    # Crear un diccionario con los resultados
    dict_partidos = {'Home': home, 'Score': score, 'Away': away}
    df_partidos = pd.DataFrame(dict_partidos)
    df_partidos["Year"] = year
    return df_partidos

# Conjunto de años para los que queremos extraer los datos
years = {1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982,
         1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018}

# Lista para almacenar los DataFrames de cada año
all_partidos_jugados = []

# Iterar sobre cada año y obtener los partidos
for year in years:
    df_partidos = get_partidos(year)
    all_partidos_jugados.append(df_partidos)

In [31]:
# Concatenar todos los DataFrames en uno solo
df_all_partidos_jugados = pd.concat(all_partidos_jugados, ignore_index=True)

# Mostrar el DataFrame final con todos los partidos
print(df_all_partidos_jugados.sort_values(by="Year").reset_index(drop=True))

             Home Score            Away  Year
0      Argentina    3–1           Chile  1930
1        Uruguay    4–2       Argentina  1930
2        Uruguay    6–1      Yugoslavia  1930
3      Argentina    6–1   United States  1930
4       Paraguay    1–0         Belgium  1930
..            ...   ...             ...   ...
859      Belgium    3–0          Panama  2018
860       Mexico    0–3          Sweden  2018
861  South Korea    2–0         Germany  2018
862  South Korea    1–2          Mexico  2018
863      Denmark    1–1       Australia  2018

[864 rows x 4 columns]


In [32]:
# Ordena el DataFrame por Year
df_all_partidos_jugados.sort_values(by="Year", inplace=True)

# Reset the index
df_all_partidos_jugados.reset_index(drop=True, inplace=True)

# Exporta el DataFrame a un archivo CSV
df_all_partidos_jugados.to_csv("all_worldcup_played_matches.csv", index=False)

In [33]:
!ls

all_worldcup_played_matches.csv  fifa_mundiales_data.csv		 sample_data
dict_tablas			 google-chrome-stable_current_amd64.deb


In [34]:
df_all_partidos_jugados

,Home,Score,Away,Year
0,Argentina,3–1,Chile,1930
1,Uruguay,4–2,Argentina,1930
2,Uruguay,6–1,Yugoslavia,1930
3,Argentina,6–1,United States,1930
4,Paraguay,1–0,Belgium,1930
...,...,...,...,...
859,Belgium,3–0,Panama,2018
860,Mexico,0–3,Sweden,2018
861,South Korea,2–0,Germany,2018
862,South Korea,1–2,Mexico,2018


In [35]:
# Lista de nombres de todas las variables que son DataFrames en el entorno actual
all_dfs = [var for var in dir() if isinstance(eval(var), pd.DataFrame)]

# Imprime el nombre de cada DataFrame y sus columnas
for df_name in all_dfs:
    df = eval(df_name)
    print(f"DataFrame: {df_name}")
    print(f"Columnas: {list(df.columns)}")
    print("-" * 40)  # Separador para mayor legibilidad

DataFrame: _
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: _10
Columnas: ['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']
----------------------------------------
DataFrame: _18
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: _34
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: _6
Columnas: ['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']
----------------------------------------
DataFrame: __
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: ___
Columnas: ['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']
----------------------------------------
DataFrame: df
Columnas: ['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']
----------------------------------------
DataFrame: df_all_partidos_jugados
Columnas: ['Home', 'Score', 'Away', 'Year']
----------

In [36]:
# Lista de DataFrames a modificar
dfs = [df_all_partidos_jugados, df_fixture2022, df_partidos, df_partidos_mundiales]

# Itera sobre cada DataFrame y aplica la modificación
for df in dfs:
    if "Home" in df.columns and "Away" in df.columns:
        df["Home"] = df["Home"].str.strip()
        df["Away"] = df["Away"].str.strip()
        df["Score"] = df["Score"].str.strip()
        df["Year"] = df["Year"].astype(int)

In [37]:
dfs[0]

,Home,Score,Away,Year
0,Argentina,3–1,Chile,1930
1,Uruguay,4–2,Argentina,1930
2,Uruguay,6–1,Yugoslavia,1930
3,Argentina,6–1,United States,1930
4,Paraguay,1–0,Belgium,1930
...,...,...,...,...
859,Belgium,3–0,Panama,2018
860,Mexico,0–3,Sweden,2018
861,South Korea,2–0,Germany,2018
862,South Korea,1–2,Mexico,2018


In [38]:
# Imprime los valores vacios de la columna "Home" de estos df
dfs = [df_all_partidos_jugados, df_fixture2022, df_partidos, df_partidos_mundiales]
for df in dfs:
    print(df[df["Home"].isnull()])  # Muestra los valores vacios de la columna "Home"
    print("-" * 40)  # Separador para mayor legibilidad

Empty DataFrame
Columns: [Home, Score, Away, Year]
Index: []
----------------------------------------
Empty DataFrame
Columns: [Home, Score, Away, Year]
Index: []
----------------------------------------
Empty DataFrame
Columns: [Home, Score, Away, Year]
Index: []
----------------------------------------
Empty DataFrame
Columns: [Home, Score, Away, Year]
Index: []
----------------------------------------


In [39]:
# Elimina los registros duplicados
df_all_partidos_jugados.drop_duplicates(inplace=True)
df_all_partidos_jugados

,Home,Score,Away,Year
0,Argentina,3–1,Chile,1930
1,Uruguay,4–2,Argentina,1930
2,Uruguay,6–1,Yugoslavia,1930
3,Argentina,6–1,United States,1930
4,Paraguay,1–0,Belgium,1930
...,...,...,...,...
859,Belgium,3–0,Panama,2018
860,Mexico,0–3,Sweden,2018
861,South Korea,2–0,Germany,2018
862,South Korea,1–2,Mexico,2018


In [40]:
# Filtra el partido "Sweden" vs "Austria" ya que fue un marcador por default y eliminalo del df_all_partidos_jugdos
df_all_partidos_jugados[df_all_partidos_jugados["Home"].str.contains("Sweden") & df_all_partidos_jugados["Away"].str.contains("Austria")]

,Home,Score,Away,Year
36,Sweden,w/o,Austria,1938


In [41]:
# Obten el index de este partido
df_all_partidos_jugados[df_all_partidos_jugados["Home"].str.contains("Sweden") & df_all_partidos_jugados["Away"].str.contains("Austria")].index

Index([36], dtype='int64')

In [42]:
# Acceder a la linea con index 47
df_all_partidos_jugados.iloc[47]

,47
Home,Cuba
Score,2–1
Away,Romania
Year,1938


In [43]:
# Filtra el DataFrame para eliminar el partido "Sweden" vs "Austria"
df_all_partidos_jugados = df_all_partidos_jugados[~(
    df_all_partidos_jugados["Home"].str.contains("Sweden") &
    df_all_partidos_jugados["Away"].str.contains("Austria")
)]

df_all_partidos_jugados.iloc[47]

,48
Home,Switzerland
Score,4–2
Away,Germany
Year,1938


In [44]:
# Filtra el df_all_partidos_jugados en donde "score" contenga datos no regulares
df_all_partidos_jugados[df_all_partidos_jugados["Score"].str.contains("[^\d–]")]

,Home,Score,Away,Year
17,Italy,2–1 (a.e.t.),Czechoslovakia,1934
24,Italy,1–1 (a.e.t.),Spain,1934
25,Austria,3–2 (a.e.t.),France,1934
35,Switzerland,1–1 (a.e.t.),Germany,1938
38,Cuba,3–3 (a.e.t.),Romania,1938
...,...,...,...,...
834,Russia,2–2 (a.e.t.),Croatia,2018
836,Croatia,2–1 (a.e.t.),England,2018
839,Colombia,1–1 (a.e.t.),England,2018
843,Croatia,1–1 (a.e.t.),Denmark,2018


In [45]:
# Obten los indices de estos registros
df_all_partidos_jugados[df_all_partidos_jugados["Score"].str.contains("[^\d–]")].index

Index([ 17,  24,  25,  35,  38,  43,  49,  50,  51,  78,  85,  92, 127, 176,
       207, 210, 212, 302, 328, 370, 377, 378, 379, 382, 414, 415, 416, 419,
       420, 422, 426, 427, 440, 442, 446, 450, 495, 496, 501, 502, 564, 565,
       568, 572, 573, 637, 640, 643, 645, 648, 664, 686, 691, 694, 700, 767,
       772, 773, 774, 778, 780, 782, 785, 834, 836, 839, 843, 844],
      dtype='int64')

In [46]:
# Reemplaza el df_all_partidos_jugados en donde "score" contenga datos no regulares
df_all_partidos_jugados["Score"] = df_all_partidos_jugados["Score"].str.replace("[^\d–]", "", regex=True)

<ipython-input-46-6e81af6e8107>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_partidos_jugados["Score"] = df_all_partidos_jugados["Score"].str.replace("[^\d–]", "", regex=True)


In [47]:
# Lista de índices a filtrar
indices = [ 17,  24,  25,  35,  38,  43,  49,  50,  51,  78,  85,  92, 127, 176,
       207, 210, 212, 302, 328, 370, 377, 378, 379, 382, 414, 415, 416, 419,
       420, 422, 426, 427, 440, 442, 446, 450, 495, 496, 501, 502, 564, 565,
       568, 572, 573, 637, 640, 643, 645, 648, 664, 686, 691, 694, 700, 767,
       772, 773, 774, 778, 780, 782, 785, 834, 836, 839, 843, 844]

# Filtra el DataFrame usando los índices proporcionados
df_all_partidos_jugados.loc[indices]

,Home,Score,Away,Year
17,Italy,2–1,Czechoslovakia,1934
24,Italy,1–1,Spain,1934
25,Austria,3–2,France,1934
35,Switzerland,1–1,Germany,1938
38,Cuba,3–3,Romania,1938
...,...,...,...,...
834,Russia,2–2,Croatia,2018
836,Croatia,2–1,England,2018
839,Colombia,1–1,England,2018
843,Croatia,1–1,Denmark,2018


In [48]:
# Filtra los registros en ["Score"] que contengan mas de 3 caracteres
df_all_partidos_jugados[df_all_partidos_jugados["Score"].str.len() > 3]

,Home,Score,Away,Year
334,Hungary,10–1,El Salvador,1982


In [49]:
import pandas as pd
import re

def validate_score_format(df):
    """
    Valida que la columna 'Score' en el DataFrame df tenga el formato '<home_goals>-<away_goals>'.
    Los formatos válidos son '#-#' o '##-##', donde # es un dígito.

    Args:
    - df (pd.DataFrame): DataFrame que contiene la columna 'Score'.

    Returns:
    - bool: True si todos los valores en 'Score' tienen el formato correcto, False en caso contrario.
    """
    # Expresión regular para validar los formatos '#-#' o '##-##'
    pattern = r'^\d{1,2}-\d{1,2}$'

    # Verifica si la columna 'Score' existe en el DataFrame
    if 'Score' not in df.columns:
        print("La columna 'Score' no existe en el DataFrame.")
        return False

    # Reemplaza guiones largos o diferentes por guion medio
    df['Score'] = df['Score'].str.replace('–', '-', regex=False)

    # Valida cada valor en la columna 'Score' contra el patrón
    valid_format = df['Score'].apply(lambda x: bool(re.match(pattern, str(x))))

    # Devuelve True si todos los valores son válidos, de lo contrario, False
    all_valid = valid_format.all()

    # Imprime los valores no válidos si hay alguno
    if not all_valid:
        invalid_values = df.loc[~valid_format, 'Score']
        print("Valores no válidos en 'Score':")
        print(invalid_values)

    return all_valid

# Ejemplo de uso con df_all_partidos_jugados
is_valid = validate_score_format(df_all_partidos_jugados)
print(f"Todos los valores en 'Score' tienen el formato correcto: {is_valid}")


Todos los valores en 'Score' tienen el formato correcto: True


<ipython-input-49-7f1880204568>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Score'] = df['Score'].str.replace('–', '-', regex=False)


In [50]:
# Asegúrate de que la columna "Score" tiene valores con el formato esperado (por ejemplo, "1-0")
# Divide la columna "Score" en dos columnas: "HomeGoals" y "AwayGoals"
# Maneja los errores donde no se encuentre el delimitador '-'
df_split = df_all_partidos_jugados["Score"].str.split("-", expand=True, n=1) # Limita el número de splits a 1

# Asegura que df_split tenga dos columnas, rellenando con NaN si es necesario
df_split = df_split.reindex(columns=[0, 1])

df_all_partidos_jugados[["HomeGoals", "AwayGoals"]] = df_split # Asigna las columnas

# Convierte las nuevas columnas a tipo numérico (opcional, si es necesario para análisis)
df_all_partidos_jugados["HomeGoals"] = pd.to_numeric(df_all_partidos_jugados["HomeGoals"], errors='coerce')
df_all_partidos_jugados["AwayGoals"] = pd.to_numeric(df_all_partidos_jugados["AwayGoals"], errors='coerce')

# Elimina la columna original "Score"
df_all_partidos_jugados.drop("Score", axis=1, inplace=True)

# Muestra las primeras filas del DataFrame para verificar los cambios
df_all_partidos_jugados.head()

<ipython-input-50-79b1604f5470>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_partidos_jugados[["HomeGoals", "AwayGoals"]] = df_split # Asigna las columnas
<ipython-input-50-79b1604f5470>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_partidos_jugados[["HomeGoals", "AwayGoals"]] = df_split # Asigna las columnas
<ipython-input-50-79b1604f5470>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,Home,Away,Year,HomeGoals,AwayGoals
0,Argentina,Chile,1930,3,1
1,Uruguay,Argentina,1930,4,2
2,Uruguay,Yugoslavia,1930,6,1
3,Argentina,United States,1930,6,1
4,Paraguay,Belgium,1930,1,0


In [51]:
# Cambia los nombres de las columnas 'Home' y 'Away' a 'HomeTeam' y 'AwayTeam' respectivamente en el DataFrame
df_all_partidos_jugados.rename(columns={"Home": "HomeTeam", "Away": "AwayTeam"}, inplace=True)

# Muestra las primeras filas del DataFrame para verificar el cambio en los nombres de las columnas
df_all_partidos_jugados.head()

<ipython-input-51-a54d2bc3cae4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_partidos_jugados.rename(columns={"Home": "HomeTeam", "Away": "AwayTeam"}, inplace=True)


,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,Argentina,Chile,1930,3,1
1,Uruguay,Argentina,1930,4,2
2,Uruguay,Yugoslavia,1930,6,1
3,Argentina,United States,1930,6,1
4,Paraguay,Belgium,1930,1,0


In [52]:
df_all_partidos_jugados.dtypes

,0
HomeTeam,object
AwayTeam,object
Year,int64
HomeGoals,int64
AwayGoals,int64


In [53]:
# Filtrar valores faltantes (NaN) en las columnas 'HomeGoals'
#df_all_partidos_jugados[df_all_partidos_jugados['HomeGoals'].isna()]
df_all_partidos_jugados[df_all_partidos_jugados['HomeGoals'].isna() | df_all_partidos_jugados['HomeGoals'].apply(lambda x: not str(x).isnumeric() if x is not None else False)]

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals


In [54]:
# Filtrar valores faltantes (NaN) o inválidos (no numéricos) en las columnas'AwayGoals'
df_all_partidos_jugados[df_all_partidos_jugados['AwayGoals'].isna() | df_all_partidos_jugados['AwayGoals'].apply(lambda x: not str(x).isnumeric() if x is not None else False)]

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals


In [55]:
df_all_partidos_jugados["TotalGoals"] = df_all_partidos_jugados["HomeGoals"] + df_all_partidos_jugados["AwayGoals"]
df_all_partidos_jugados.head()

<ipython-input-55-f3b5135afd7d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_partidos_jugados["TotalGoals"] = df_all_partidos_jugados["HomeGoals"] + df_all_partidos_jugados["AwayGoals"]


,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,Argentina,Chile,1930,3,1,4
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1


In [56]:
df_all_partidos_jugados.to_csv("clean_all_fifa_worldcup_played_matches.csv", index=False)
df_fixture2022.to_csv("fifa_worldcup_fixture_2022.csv", index=False)

Ya has limpiado los datos, ahora falta la prediccion

url = https://www.youtube.com/watch?v=EuZ4gwNNYwg

timestamp: 02:15:42

In [57]:
from scipy.stats import poisson

In [58]:
!ls

all_worldcup_played_matches.csv		    fifa_mundiales_data.csv		    sample_data
clean_all_fifa_worldcup_played_matches.csv  fifa_worldcup_fixture_2022.csv
dict_tablas				    google-chrome-stable_current_amd64.deb


In [60]:
# Lista de nombres de todas las variables que son DataFrames en el entorno actual
all_dfs = [var for var in dir() if isinstance(eval(var), pd.DataFrame)]

# Imprime el nombre de cada DataFrame y sus columnas
for df_name in all_dfs:
    df = eval(df_name)
    print(f"DataFrame: {df_name}")
    print(f"Columnas: {list(df.columns)}")
    print("-" * 40)  # Separador para mayor legibilidad

DataFrame: _
Columnas: ['HomeTeam', 'AwayTeam', 'Year', 'HomeGoals', 'AwayGoals', 'TotalGoals']
----------------------------------------
DataFrame: _10
Columnas: ['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']
----------------------------------------
DataFrame: _18
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: _34
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: _37
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: _39
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: _40
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: _44
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: _47
Columnas: ['Home', 'Score', 'Away', 'Year']
----------------------------------------
DataFrame: _48
Columnas: ['Home', 

In [68]:
# Cargar el diccionario de tablas desde un archivo pickle
# dict_tablas = pickle.load(open('dict_tablas', 'rb'))
# dict_tablas.keys()
# dict_tablas['Grupo C']

#Carga el fixture de 2022
#df_fixture2022

# Carga todos los partidos jugados
#df_all_partidos_jugados


,Home,Score,Away,Year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


# Calcular la fuerza por equipos

In [69]:
df_all_partidos_jugados.tail()

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
859,Belgium,Panama,2018,3,0,3
860,Mexico,Sweden,2018,0,3,3
861,South Korea,Germany,2018,2,0,2
862,South Korea,Mexico,2018,1,2,3
863,Denmark,Australia,2018,1,1,2


In [70]:
# Dividir el dataframe df_all_partidos_jugados en df_home y df_away
df_home = df_all_partidos_jugados[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_all_partidos_jugados[['AwayTeam', 'AwayGoals', 'HomeGoals']]

In [73]:
# Renombrar columnas en df_home para estandarizar nombres (equipo, goles anotados, goles recibidos)
df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'}, inplace=True)

# Renombrar columnas en df_away para estandarizar nombres (equipo, goles anotados, goles recibidos)
df_away.rename(columns={'AwayTeam': 'Team', 'AwayGoals': 'GoalsScored', 'HomeGoals': 'GoalsConceded'}, inplace=True)

<ipython-input-73-f143c08ac988>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'}, inplace=True)
<ipython-input-73-f143c08ac988>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_away.rename(columns={'AwayTeam': 'Team', 'AwayGoals': 'GoalsScored', 'HomeGoals': 'GoalsConceded'}, inplace=True)


In [75]:
# Revisar los dataframes
df_home
#df_away

,Team,GoalsScored,GoalsConceded
0,Argentina,3,1
1,Uruguay,4,2
2,Uruguay,6,1
3,Argentina,6,1
4,Paraguay,1,0
...,...,...,...
859,Belgium,3,0
860,Mexico,0,3
861,South Korea,2,0
862,South Korea,1,2


In [77]:
# Combinar df_home y df_away, luego agrupar por equipo y calcular la media de las estadísticas
df_team_strenght = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()

# Mostrar el DataFrame resultante con la fortaleza promedio de cada equipo
df_team_strenght

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.717949,1.166667
Australia,0.812500,1.937500
Austria,1.576923,1.692308
...,...,...
Uruguay,1.603774,1.339623
Wales,0.800000,0.800000
West Germany,2.050847,1.254237


In [78]:
# agregar una columna que cuente la cantidad de partidos jugados por cada equipo
df_team_strenght['MatchesPlayed'] = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').size()
df_team_strenght

,GoalsScored,GoalsConceded,MatchesPlayed
Team,,,
Algeria,1.000000,1.461538,13
Angola,0.333333,0.666667,3
Argentina,1.717949,1.166667,78
Australia,0.812500,1.937500,16
Austria,1.576923,1.692308,26
...,...,...,...
Uruguay,1.603774,1.339623,53
Wales,0.800000,0.800000,5
West Germany,2.050847,1.254237,59


In [82]:
# Calcular el número de partidos jugados por cada equipo en casa y fuera
df_home['MatchesPlayed'] = df_home.groupby('Team')['Team'].transform('count')
df_away['MatchesPlayed'] = df_away.groupby('Team')['Team'].transform('count')

# Calcular la fuerza de ataque promedio por partido
df_home['AttackingStrength'] = df_home['GoalsScored'] / df_home['MatchesPlayed']
df_away['AttackingStrength'] = df_away['GoalsScored'] / df_away['MatchesPlayed']

# Calcular la fuerza defensiva promedio por partido (goles concedidos por partido)
df_home['DefensiveStrength'] = df_home['GoalsConceded'] / df_home['MatchesPlayed']
df_away['DefensiveStrength'] = df_away['GoalsConceded'] / df_away['MatchesPlayed']

In [83]:
df_home

,Team,GoalsScored,GoalsConceded,AttackingStrength,DefensiveStrength,MatchesPlayed
0,Argentina,3,1,0.054545,0.018182,55
1,Uruguay,4,2,0.129032,0.064516,31
2,Uruguay,6,1,0.193548,0.032258,31
3,Argentina,6,1,0.109091,0.018182,55
4,Paraguay,1,0,0.090909,0.000000,11
...,...,...,...,...,...,...
859,Belgium,3,0,0.157895,0.000000,19
860,Mexico,0,3,0.000000,0.166667,18
861,South Korea,2,0,0.142857,0.000000,14
862,South Korea,1,2,0.071429,0.142857,14


In [114]:
# Funcion redactada por un humano
def predict_points(home_away):
    home, away = home_away

    # Verificación de que ambos equipos estén en la lista
    if home not in df_team_strenght.index:
        raise ValueError(f"El equipo local '{home}' no está en la lista de equipos. Por favor, elige un equipo válido.")
    if away not in df_team_strenght.index:
        raise ValueError(f"El equipo visitante '{away}' no está en la lista de equipos. Por favor, elige un equipo válido.")

    lamb_home = df_team_strenght.at[home, "GoalsScored"] * df_team_strenght.at[away, "GoalsConceded"]
    lamb_away = df_team_strenght.at[away, "GoalsScored"] * df_team_strenght.at[home, "GoalsConceded"]
    prob_home, prob_away, prob_draw = 0, 0, 0

    for x in range(0, 11):
        for y in range(0, 11):
            p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
            if x == y:
                prob_draw += p
            elif x > y:
                prob_home += p
            else:
                prob_away += p

    points_home = 3 * prob_home + prob_draw
    points_away = 3 * prob_away + prob_draw
    return (points_home, points_away)

In [115]:
from scipy.stats import poisson

# Funcion redactada por ChatGPT
def predict_points_GPT(home_away):
    home, away = home_away

    # Verificación de que ambos equipos estén en la lista
    if home not in df_team_strenght.index:
        raise ValueError(f"El equipo local '{home}' no está en la lista de equipos. Por favor, elige un equipo válido.")
    if away not in df_team_strenght.index:
        raise ValueError(f"El equipo visitante '{away}' no está en la lista de equipos. Por favor, elige un equipo válido.")

    lamb_home = df_team_strenght.at[home, "GoalsScored"] * df_team_strenght.at[away, "GoalsConceded"]
    lamb_away = df_team_strenght.at[away, "GoalsScored"] * df_team_strenght.at[home, "GoalsConceded"]

    # Reducir el rango a goles más probables (0 a 5)
    max_goals = 5

    # Precalcular las probabilidades de goles para ambos equipos
    prob_home_goals = [poisson.pmf(x, lamb_home) for x in range(max_goals + 1)]
    prob_away_goals = [poisson.pmf(y, lamb_away) for y in range(max_goals + 1)]

    prob_home, prob_away, prob_draw = 0, 0, 0

    for x in range(max_goals + 1):
        for y in range(max_goals + 1):
            p = prob_home_goals[x] * prob_away_goals[y]
            if x == y:
                prob_draw += p
            elif x > y:
                prob_home += p
            else:
                prob_away += p

    points_home = 3 * prob_home + prob_draw
    points_away = 3 * prob_away + prob_draw
    return (points_home, points_away)

In [116]:
from scipy.stats import poisson

def predict_points_GPT2(home_away):
    home, away = home_away

    # Verificación de que ambos equipos estén en la lista
    if home not in df_team_strenght.index or away not in df_team_strenght.index:
        return (0, 0)  # Si alguno de los equipos no tiene datos, se asigna 0 puntos a ambos

    lamb_home = df_team_strenght.at[home, "GoalsScored"] * df_team_strenght.at[away, "GoalsConceded"]
    lamb_away = df_team_strenght.at[away, "GoalsScored"] * df_team_strenght.at[home, "GoalsConceded"]

    # Reducir el rango a goles más probables (0 a 5)
    max_goals = 5

    # Precalcular las probabilidades de goles para ambos equipos
    prob_home_goals = [poisson.pmf(x, lamb_home) for x in range(max_goals + 1)]
    prob_away_goals = [poisson.pmf(y, lamb_away) for y in range(max_goals + 1)]

    prob_home, prob_away, prob_draw = 0, 0, 0

    for x in range(max_goals + 1):
        for y in range(max_goals + 1):
            p = prob_home_goals[x] * prob_away_goals[y]
            if x == y:
                prob_draw += p
            elif x > y:
                prob_home += p
            else:
                prob_away += p

    points_home = 3 * prob_home + prob_draw
    points_away = 3 * prob_away + prob_draw
    return (points_home, points_away)

In [102]:
# Crea un input de una letra y se devuelva todos los equipos presentes en el

letra = input("Ingresa una letra: ").upper()

# Filtrar el DataFrame para mostrar solo los equipos que comienzan con la letra ingresada
equipos_filtrados = df_team_strenght[df_team_strenght.index.str.startswith(letra)]

if equipos_filtrados.empty:
    print(f"No hay equipos que comiencen con la letra '{letra}'.")
else:
    print(f"Equipos que comienzan con la letra '{letra}':")
    print(equipos_filtrados)

Ingresa una letra: U
Equipos que comienzan con la letra 'U':
               GoalsScored  GoalsConceded  MatchesPlayed
Team                                                    
Ukraine           1.000000       1.400000              5
United States     1.166667       1.800000             30
Uruguay           1.603774       1.339623             53


In [123]:
# Definir los equipos en una variable
teams_to_evaluate = ["Qatar (H)", "Uruguay"]

In [124]:
# evalua teams_to_evaluate con la funcion predict_points
predict_points(teams_to_evaluate)

ValueError: El equipo local 'Qatar' no está en la lista de equipos. Por favor, elige un equipo válido.

In [125]:
# Llama a la funcion predict_pointsGPT con teams_to_evaluate
predict_points_GPT(teams_to_evaluate)

ValueError: El equipo local 'Qatar' no está en la lista de equipos. Por favor, elige un equipo válido.

In [126]:
# Llama a la funcioon predict_points_GPT2 con teams_to_evlauate
predict_points_GPT2(teams_to_evaluate)

(0, 0)

In [175]:
print(df_fixture2022)

                Home     Score              Away  Year
0          Qatar (H)   Match 1           Ecuador  2022
1            Senegal   Match 2       Netherlands  2022
2          Qatar (H)  Match 18           Senegal  2022
3        Netherlands  Match 19           Ecuador  2022
4            Ecuador  Match 35           Senegal  2022
..               ...       ...               ...   ...
59  Winners Match 51  Match 59  Winners Match 52  2022
60  Winners Match 57  Match 61  Winners Match 58  2022
61  Winners Match 59  Match 62  Winners Match 60  2022
62   Losers Match 61  Match 63   Losers Match 62  2022
63  Winners Match 61  Match 64  Winners Match 62  2022

[64 rows x 4 columns]


In [176]:
# Dividir el df_fixture2022 en fase de grupos, 16vos, 8vos, 4tos, Semi y Final
df_fixture2022_group_stage = df_fixture2022[:48].copy()
df_fixture2022_round16 = df_fixture2022[48:56].copy()
df_fixture2022_quarter_finals = df_fixture2022[56:60].copy()
df_fixture2022_semi_finals = df_fixture2022[60:62].copy()
df_fixture2022_final = df_fixture2022[62:].copy()

In [177]:
df_fixture2022_round16

,Home,Score,Away,Year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [178]:
!ls

all_worldcup_played_matches.csv		    fifa_mundiales_data.csv		    sample_data
clean_all_fifa_worldcup_played_matches.csv  fifa_worldcup_fixture_2022.csv
dict_tablas				    google-chrome-stable_current_amd64.deb


In [179]:
for group in dict_tablas:
    print(dict_tablas[group]["Team"].values)

['Netherlands' 'Senegal' 'Ecuador' 'Qatar (H)']
['England' 'Wales' 'United States' 'Iran']
['Argentina' 'Poland' 'Mexico' 'Saudi Arabia']
['France' 'Denmark' 'Tunisia' 'Australia']
['Germany' 'Spain' 'Japan' 'Costa Rica']
['Croatia' 'Belgium' 'Morocco' 'Canada']
['Brazil' 'Switzerland' 'Serbia' 'Cameroon']
['Portugal' 'Uruguay' 'Ghana' 'South Korea']


In [180]:
dict_tablas.keys()

dict_keys(['Grupo A', 'Grupo B', 'Grupo C', 'Grupo D', 'Grupo E', 'Grupo F', 'Grupo G', 'Grupo H'])

In [181]:
dict_tablas["Grupo A"]

,Team,Pts
0,Netherlands,16.0
1,Senegal,9.0
2,Ecuador,8.0
3,Qatar (H),0.0


In [182]:
df_fixture2022_group_stage[df_fixture2022_group_stage["Home"].isin(['Qatar (H)', 'Ecuador', 'Senegal', 'Netherlands'])]

,Home,Score,Away,Year
0,Qatar (H),Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar (H),Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar (H),2022


In [183]:
# Asegúrate de que dict_grupos tenga las columnas adecuadas
for group in dict_tablas.keys():
    teams_in_group = dict_tablas[group]["Team"].values
    df_fix_group_6 = df_fixture2022_group_stage[df_fixture2022_group_stage["Home"].isin(teams_in_group)]

    # Iterar sobre los partidos en el grupo con la funcion predict_points_GPT2
    for index, row in df_fix_group_6.iterrows():
        home, away = row["Home"], row["Away"]
        points_home, points_away = predict_points_GPT2([home, away])

        # Actualizar los puntos en dict_tablas
        dict_tablas[group].loc[dict_tablas[group]["Team"] == home, "Pts"] += points_home
        dict_tablas[group].loc[dict_tablas[group]["Team"] == away, "Pts"] += points_away

    # Ordenar los equipos del grupo por puntos y actualizar dict_tablas
    dict_tablas[group] = dict_tablas[group].sort_values("Pts", ascending=False).reset_index(drop=True)
    dict_tablas[group] = dict_tablas[group][["Team", "Pts"]]
    dict_tablas[group] = dict_tablas[group].round(0)

In [184]:
dict_tablas["Grupo B"]

,Team,Pts
0,England,24.0
1,Wales,20.0
2,United States,16.0
3,Iran,8.0


In [185]:
dict_tablas["Grupo C"]

,Team,Pts
0,Argentina,24.0
1,Poland,20.0
2,Mexico,12.0
3,Saudi Arabia,4.0


In [186]:
dict_tablas["Grupo D"]

,Team,Pts
0,France,24.0
1,Denmark,20.0
2,Tunisia,12.0
3,Australia,8.0


In [187]:
dict_tablas["Grupo E"]

,Team,Pts
0,Germany,24.0
1,Spain,20.0
2,Japan,12.0
3,Costa Rica,8.0


In [188]:
dict_tablas["Grupo F"]

,Team,Pts
0,Croatia,24.0
1,Belgium,20.0
2,Morocco,16.0
3,Canada,0.0


In [189]:
dict_tablas["Grupo G"]

,Team,Pts
0,Brazil,24.0
1,Switzerland,16.0
2,Serbia,12.0
3,Cameroon,8.0


In [190]:
dict_tablas["Grupo H"]

,Team,Pts
0,Portugal,20.0
1,Uruguay,20.0
2,Ghana,16.0
3,South Korea,8.0


In [191]:
df_fixture2022_round16

,Home,Score,Away,Year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [192]:
# Actualizar el df_fixture2022_round16 con los grupos Winners y Runners-up de la fase de grupos
for group in dict_tablas:
    group_winner = dict_tablas[group].loc[0, "Team"]
    runners_up = dict_tablas[group].loc[1, "Team"]

    # Convertir el nombre del grupo al formato "Group X"
    group_english = group.replace("Grupo", "Group")

    df_fixture2022_round16.replace({f"Winners {group_english}": group_winner,
                                    f"Runners-up {group_english}": runners_up}, inplace=True)

df_fixture2022_round16["Winner"] = "?"
df_fixture2022_round16

,Home,Score,Away,Year,Winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [205]:
def predict_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row["Home"], row["Away"]

        # Pasar una tupla (home, away) a predict_points_GPT2
        points_home, points_away = predict_points_GPT2((home, away))

        if points_home > points_away:
            df_fixture_updated.loc[index, "Winner"] = home
        else:
            df_fixture_updated.loc[index, "Winner"] = away

    return df_fixture_updated

In [206]:
predict_winner(df_fixture2022_round16)

,Home,Score,Away,Year,Winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Croatia
55,Portugal,Match 56,Switzerland,2022,Portugal


In [207]:
df_fixture2022_quarter_finals

,Home,Score,Away,Year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [208]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, "Winner"]
        match = df_fixture_round_1.loc[index, "Score"]

        # Reemplazar "Winners Match X" en df_fixture_round_2 con el ganador correspondiente
        df_fixture_round_2.replace({f'Winners {match}': winner}, inplace=True)

    df_fixture_round_2["Winner"] = "?"
    return df_fixture_round_2

In [210]:
# debes llamar a la función update_table con ambos DataFrames
update_table(df_fixture2022_round16, df_fixture2022_quarter_finals)

,Home,Score,Away,Year,Winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Croatia,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [212]:
predict_winner(df_fixture2022_quarter_finals)

,Home,Score,Away,Year,Winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Croatia,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


In [213]:
# Semifinal
update_table(df_fixture2022_quarter_finals, df_fixture2022_semi_finals)

,Home,Score,Away,Year,Winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [215]:
predict_winner(df_fixture2022_semi_finals)  # Semifinal

,Home,Score,Away,Year,Winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


In [216]:
# Final
update_table(df_fixture2022_semi_finals, df_fixture2022_final)

,Home,Score,Away,Year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [217]:
# Ganador
predict_winner(df_fixture2022_final)

,Home,Score,Away,Year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
